In [1]:
import os, sys
import numpy as np
import pandas as pd
import gmms

# Example: 1989 Loma Prieta Earthquake

## Earthquake parameters

In [2]:
M      = 6.93
hyp_Z  = 17.48
region = 'Global' # regionalized models for 'Japan', 'Italy', and 'China'

## Fault model

In [3]:
ULC_lat = 37.19
ULC_lon = -122.03
flength = 40.
fwidth  = 18
fstrike = 128.
fdip    = 70.
frake   = 140.
fZtor   = 3.85

## Site parameters

In [4]:
site_lat  = 37.825
site_lon  = -122.373
site_Vs30 = 155.11

## Convert to arrays

In [5]:
vals = []
for val in zip([ULC_lat,ULC_lon,flength,fwidth,fstrike,fdip,frake,fZtor,site_lat,site_lon,site_Vs30]):
    if isinstance(val[0],np.ndarray):
        vals.append(val[0])
    elif isinstance(val[0], (int, float)):
        vals.append(np.array([float(val[0])]))
    elif isinstance(val[0], (list, tuple)):
        vals.append(np.array(val[0], dtype=float))
    else:
        vals.append(val[0])
[ULC_lat,ULC_lon,flength,fwidth,fstrike,fdip,frake,fZtor,site_lat,site_lon,site_Vs30] = vals

## Compute fault parameters

In [6]:
[URC_lat,URC_lon] = gmms.get_fault_URC(ULC_lat,ULC_lon,flength,fstrike)
[fnm,frv,fmech]   = gmms.get_faulting_style(frake)

## Compute site parameters

In [7]:
site_Z1p0 = np.exp((-7.15/4)*np.log((site_Vs30**4+571**4)/(1360**4+571**4))) # BSSA14
site_Z2p5 = np.exp(7.089-1.144*np.log(site_Vs30))                            # CB13

In [8]:
[R_JB,R_rup,R_x] = gmms.get_distances_cy(site_lat,site_lon,ULC_lat,ULC_lon,URC_lat,URC_lon,fwidth,flength,fdip,fstrike,fZtor)

In [9]:
print('Rjb  = {:.2f} km'.format(R_JB[0]))
print('Rrup = {:.2f} km'.format(R_rup[0]))
print('Rx   = {:.2f} km'.format(R_x[0]))

Rjb  = 76.72 km
Rrup = 76.82 km
Rx   = -37.00 km


## Compute ground motions

### CAVgm

In [10]:
[mu,phi,tau] = gmms.CampbellBozorgnia2010.CampbellBozorgnia2010_cy(M,fdip[0],fZtor[0],R_JB,R_rup,R_x,site_Vs30,site_Z2p5,fnm,frv)
print('CAVstd = {:.2f} m/s'.format(np.exp(mu[0])))

CAVstd = 0.53 m/s


### CAVdp

In [11]:
flag_CAVstd = 1 # '0' for recorded, '1' for gmm-based
flag_PSV    = 0 # '0' for no PSV check, '1' otherwise

In [12]:
[mu,phi,tau] = gmms.CampbellBozorgnia2011.CampbellBozorgnia2011_cy(M,fdip[0],fZtor[0],R_JB,R_rup,R_x,site_Vs30,site_Z1p0,site_Z2p5,fnm,frv,flag_CAVstd,flag_PSV)
print('CAVdp = {:.2f} m/s'.format(np.exp(mu[0])))

CAVdp = 0.37 m/s


### Arias Intensity

In [13]:
[mu,phi,tau] = gmms.CampbellBozorgnia2019.CampbellBozorgnia2019_cy(0,M,fwidth[0],fdip[0],fZtor[0],hyp_Z,R_JB,R_rup,R_x,site_Vs30,site_Z2p5,fnm,frv,region)
print('Ia = {:.2f} m/s'.format(np.exp(mu[0])))

Ia = 0.20 m/s


### Cumulative absolute velocity

In [14]:
[mu,phi,tau] = gmms.CampbellBozorgnia2019.CampbellBozorgnia2019_cy(1,M,fwidth[0],fdip[0],fZtor[0],hyp_Z,R_JB,R_rup,R_x,site_Vs30,site_Z2p5,fnm,frv,region)
print('CAV = {:.2f} m/s'.format(np.exp(mu[0])))

CAV = 5.53 m/s
